# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto - Week 3 Assessment

First, it is necessary to import all the required dependencies and libraries

In [106]:
#import
import pandas as pd
from bs4 import BeautifulSoup
import requests
import collections
#file = open(filename, encoding="utf8")
#Alternatively, if we want to feed the webpage directly we can do the following:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
newsoup = BeautifulSoup(source, 'lxml')
#print(newsoup.prettify())

In [107]:
tbl = newsoup.find('tbody') #find the table body element where all the information is embedded

trs= tbl.find_all('tr') #find all the table rows containing the information
#print(trs[0])

#from the first table row trs[0], obtain the table headings and save them in an array of strings
titles=trs[0].find_all('th')
title1=titles[0].text
title2=titles[1].text
title3=titles[2].text
title3=title3.split('\n')[0]
titles=[title1,title2,title3] #these are the titles for the dataframe colums
print(titles)

['Postcode', 'Borough', 'Neighbourhood']


In [126]:

info = []
postcodes=[]
boroughs=[]
neighbourhoods=[]

for x in range(1,len(trs)):  #this would be the beginning of the code needed to iterate over the rest of the rows

#trying with the first row containing valuable informtaion:
    tds=trs[x].find_all('td') #from the row, find all the tds
    cell1=tds[0].text
    cell2=tds[1].text
    cell3=tds[2].text.split('\n')[0]
    if cell2 != 'Not assigned':
        if cell3 == 'Not assigned':
            cell3=cell2
        row1=[cell1,cell2,cell3]
        postcodes.append(cell1) #append the the first cell of the row (Postcode) to the postcodes list
        boroughs.append(cell2) #append the second cell of the row (Borough) to the boroughs list
        neighbourhoods.append(cell3) #append the third cell of the row (Neighbourhood) to the neighbourhoods list
        
        info.append(row1) #create a list with lists containing the rows of the Postcodes' table and append information
        
#print(info) #in case someone wants to look at the information collected from the website

columnas={'pcode':postcodes,'boroughs':boroughs,'neighbourhoods':neighbourhoods}
df=pd.DataFrame(columnas)

len(df.pcode.unique())

df = df.groupby(['pcode','boroughs'])['neighbourhoods'].apply(', '.join).reset_index()

df.head()

df.loc[df['pcode'] == 'M5A']


,pcode,boroughs,neighbourhoods
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


Then, we start scraping the Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 

One of the most commong packages is BeautifulSoup (http://beautiful-soup-4.readthedocs.io/en/latest/)


When processing the information to produce a proper dataframe, it is important to consider the following

   <ul>
      <li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</li>
      <li>Only process the cells that have an assigned borough</li>
        <ul>
            <li>Ignore cells with a borough that is <strong>Not assigned</strong>.</li>
        </ul>
      <li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</li>
    </ul>